In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/processed/processed.csv')

In [3]:
df.shape

(5191, 9)

In [4]:
df.sample(3)

,kms_driven,owner,location,mileage,power,price,brand,engine,age
4584,45000.0,first,mumbai,55.0,14.0,30000.0,Hero,150.0,8
324,58000.0,first,other,35.0,25.0,95000.0,KTM,200.0,6
1666,72000.0,first,other,40.0,19.8,91600.0,Royal Enfield,350.0,7


In [5]:
X = df.drop('price',axis=1)
y = df['price']

In [6]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,17000.0,first,hyderabad,35.0,19.0,Bajaj,NaN,4
1,50000.0,first,hyderabad,35.0,19.8,Royal Enfield,350.0,5
2,14795.0,first,hyderabad,30.0,28.0,other,NaN,9
3,24561.0,third,bangalore,35.0,25.0,KTM,200.0,9
4,19718.0,first,bangalore,65.0,17.0,Bajaj,180.0,5


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=11)

In [8]:
y = y.apply(np.log1p)

## Category Encoding

In [9]:
X.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
0,17000.0,first,hyderabad,35.0,19.0,Bajaj,NaN,4
1,50000.0,first,hyderabad,35.0,19.8,Royal Enfield,350.0,5
2,14795.0,first,hyderabad,30.0,28.0,other,NaN,9
3,24561.0,third,bangalore,35.0,25.0,KTM,200.0,9
4,19718.0,first,bangalore,65.0,17.0,Bajaj,180.0,5


## Impute missing values

In [10]:
from sklearn.impute import KNNImputer

num_cols = [ col for col in X.columns if X[col].dtypes != 'object' ]
print(num_cols)

['kms_driven', 'mileage', 'power', 'engine', 'age']


In [11]:
imputer = KNNImputer(n_neighbors=7,weights='distance')

imputer.fit(X_train[num_cols])

# X_train[num_cols] = imputer.transform(X_train[num_cols])
# X_test[num_cols] = imputer.transform(X_test[num_cols])
# imputer.fit(X_train[num_cols])

X_train.loc[:][num_cols] = imputer.fit_transform(X_train[num_cols])
X_test.loc[:][num_cols] = imputer.transform(X_test[num_cols])

# print(X_train.shape,y_train.shape, X_test.shape,y_test.shape)

In [12]:
print('Missing Values in Train set',X_train.isnull().sum())
print('Missing Values in Test set',X_test.isnull().sum())

Missing Values in Train set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64
Missing Values in Test set kms_driven    0
owner         0
location      0
mileage       0
power         0
brand         0
engine        0
age           0
dtype: int64


In [13]:
X_train.head()

,kms_driven,owner,location,mileage,power,brand,engine,age
4654,4130.0,first,mumbai,30.683956,38.26,other,333.908883,4
2716,7800.0,first,other,35.000000,19.10,Royal Enfield,350.000000,1
48,9500.0,first,hyderabad,36.390338,26.10,Suzuki,250.000000,2
2686,26000.0,first,other,38.000000,21.00,Bajaj,333.361084,4
4851,40997.0,first,other,63.000000,14.00,Suzuki,150.000000,6


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,MinMaxScaler

category_transformer = ColumnTransformer([
    ("kms_driven_engine_min_max_scaler",MinMaxScaler(),[0,6,3,4]),
    ("owner_ordinal_enc",OrdinalEncoder(categories=[['fourth','third','second','first']],handle_unknown='ignore',dtype=np.int16),[1]),
    ("brand_location_ohe",OneHotEncoder(sparse=False,handle_unknown='error',drop='first',),[2,5]),
],remainder='passthrough')



## Model Building

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn import set_config

set_config(display='diagram')

In [16]:

def build_pipeline_with_estimator(estimator):
    return Pipeline([
    ('category_transformer',category_transformer),
    ('estimator',estimator),
])


In [17]:
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
# MAPE
def mape(targets, predictions):
    return np.mean(np.abs((targets - predictions)) / targets) * 100

# Adjusted R^2
def adj_r2(ind_vars, targets, predictions):
    r2 = r2_score(targets, predictions)
    n = ind_vars.shape[0]
    k = ind_vars.shape[1]
    return 1-((1-r2)*(n-1)/(n-k-1))

# Model performance check
def model_perf(model, inp, out,cross_val=True):

    y_pred = model.predict(inp)
    y_act = out.values

    cross_val_ = cross_val_score(model, inp,out,cv=10).mean() if cross_val else None
    

    return pd.DataFrame({
                "RMSE": np.sqrt(mean_squared_error(y_act, y_pred)),
                "MAE": mean_absolute_error(y_act, y_pred),
                "MAPE": mape(y_act, y_pred),
                "R^2": r2_score(y_act, y_pred),
                "Adjusted R^2": adj_r2(inp, y_act, y_pred),
                "Cross Val Score (Mean)": cross_val_ if cross_val else None
           }, index=[0])

## LinearRegression

In [18]:
linear_regressor = build_pipeline_with_estimator(LinearRegression())

linear_regressor.fit(X_train,y_train)

print('Linear Regression Train Performance.\n')
print(model_perf(linear_regressor,X_train,y_train,True))

print('Linear Regression Test Performance.\n')
print(model_perf(linear_regressor,X_test,y_test,True))

Linear Regression Train Performance.

          RMSE           MAE      MAPE      R^2  Adjusted R^2  \
0  63555.51903  29371.225965  41.46409  0.75494      0.754435   

   Cross Val Score (Mean)  
0                0.738938  
Linear Regression Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  52265.136849  27641.334245  44.509878  0.836176      0.835159   

   Cross Val Score (Mean)  
0                0.824122  


## RandomForest

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

model = build_pipeline_with_estimator(RandomForestRegressor())

params = {
    'estimator__criterion': ['mse','mae'],
    'estimator__n_estimators': [100,110,120,130],
    'estimator__max_depth': [5,10,15,20,25,30],
    'estimator__min_samples_split': range(2,20),
    'estimator__max_features': ['auto','sqrt','log2'],
}
model = GridSearchCV(model,params,cv=5)
# model = RandomizedSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('RandomForest Train Performance.\n')
print(model_perf(model,X_train,y_train,False))

print('RandomForest Test Performance.\n')
print(model_perf(model,X_test,y_test,False))

KeyboardInterrupt: 

## XGBoost

In [ ]:
from xgboost import XGBRegressor

xgboost = build_pipeline_with_estimator(XGBRegressor())

xgboost.fit(X_train,y_train)

print('xgboost Train Performance.\n')
print(model_perf(xgboost,X_train,y_train))

print('xgboost Test Performance.\n')
print(model_perf(xgboost,X_test,y_test))

xgboost Train Performance.

           RMSE         MAE      MAPE       R^2  Adjusted R^2  \
0  11192.012239  7575.32473  11.02311  0.992401      0.992385   

   Cross Val Score (Mean)  
0                0.754148  
xgboost Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  49860.029678  19263.364292  28.640937  0.850906      0.849981   

   Cross Val Score (Mean)  
0                0.782039  


## KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

model = build_pipeline_with_estimator(KNeighborsRegressor())

# params = {
#     'estimator__n_neighbors': [3,5,7,9],
#     'estimator__algorithm': ['ball_tree','kd_tree','brute'],
#     'estimator__leaf_size': [5,10,30,32,35]
# }

# model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('KNeighborsRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('KNeighborsRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

KNeighborsRegressor Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  63960.163058  22412.642975  22.012028  0.751809      0.751298   

   Cross Val Score (Mean)  
0                0.569198  
KNeighborsRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  73835.925684  26567.700331  28.876153  0.673043      0.671014   

   Cross Val Score (Mean)  
0                0.572433  


## GradientBoostingRegressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model = build_pipeline_with_estimator(GradientBoostingRegressor())

# params = {
#     'estimator__loss': ['ls','lad','huber','quantile'],
#     'estimator__learning_rate': [0.1,0.01,0.001],
#     'estimator__n_estimators': [100,110,120],
#     'estimator__criterion': ['friedman_mse','mse'],
# }

# model = GridSearchCV(model,params,cv=5)

model.fit(X_train,y_train)

print('GradientBoostingRegressor Train Performance.\n')
print(model_perf(model,X_train,y_train))

print('GradientBoostingRegressor Test Performance.\n')
print(model_perf(model,X_test,y_test))

GradientBoostingRegressor Train Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  36125.442992  17711.728825  22.755422  0.920824      0.920661   

   Cross Val Score (Mean)  
0                0.777618  
GradientBoostingRegressor Test Performance.

           RMSE           MAE       MAPE       R^2  Adjusted R^2  \
0  35574.074916  18531.486678  26.153539  0.924103      0.923632   

   Cross Val Score (Mean)  
0                0.834145  
